# FINAL PROJECT: FUNCTIONAL PROGRAMMING E-COMMERCE SYSTEM

### Isabel de Paula Diez de Rivera Vergara - NIA : 100429935@alumnos.uc3m.es
### Ivan Ocheretianyi - NIA: 100487317@alumnos.uc3m.es

## Introduction

This project builds a small e-commerce system in Haskell. It models the main parts of an online shop using well-defined types: **Category** and **Product** describe what the shop sells, **Customer** and **LoyaltyLevel** show who is buying and which discounts apply, **CartItem** and **ShoppingCart** represent what the customer wants to buy, **Stock** stores available quantities, and **Order** combines all of this with a **Status**.

Discounts are handled through the *Discountable* type class. Both **LoyaltyLevel** and **Category** are instances of this class, so the final price of each item is obtained by applying both discounts in order. The function `calculateOrderTotal` then adds everything together.

The system also checks correctness. Equality for **Product** and **Customer** is based on their identifiers, stock is verified through `stockQuantity` and `checkStock`, and `createOrder` ensures that orders with missing items are rejected.

Order progress is controlled by the **Status** type. The function `transitionAllowed` defines which changes are valid, and `updateOrderStatus` returns a new order only when the transition is permitted, using `Maybe` to represent success or failure.

The project also includes a small search module. Orders can be filtered using different criteria such as customer id, loyalty level, product id, category, or total price. High-value customers are identified using helper functions like `activeOrders`, `customersOf`, `uniq`, and `totalOfCustomer`.

Finally, these pure functions are meant to be combined with I/O so users can search products, manage carts, place orders, and so that shop owners can inspect customers and process pending orders.

## CODE

1. Exhaustive Type Definition: Define exhaustively all the former concepts, using the most appropriate way (type, data, newtype) for each of them. Ensure that aliases are defined as needed to make arguments of the new types easier to understand.
2. Pretty Printing Functions: Functions to show the former concepts in a clear and readable way by implementing the Show Type Class.

+ **Category** is a custom `data` type that represents the type of product sold in the shop. It includes four fixed options: `Electronics`, `Books`, `Clothing`, and `Groceries`. The type uses `deriving (Show, Eq)` so that values can be printed and compared automatically.
<br><br>
+ **Product** is a custom `data` type that stores information about each item in the shop. It has four fields:
  + `pid` of type `Int` for the product’s unique id,
  + `pname` of type `String` for the product name,
  + `price` of type `Float`,
  + and `category` of type **Category**.
  A custom `Show` instance is defined so that products are printed in a clear and readable way.
<br><br>
+ **LoyaltyLevel** is a `data` type describing the customer’s loyalty status: `Bronze`, `Silver`, or `Gold`. It uses `deriving (Show, Eq)` to allow printing and equality checks.
<br><br>
+ **Customer** is a custom `data` type storing information about a client. It contains:
  + `cid` of type `Int` as the customer id,
  + `cname` of type `String` for the customer’s name,
  and a loyalty level of type **LoyaltyLevel**.
  A custom `Show` instance prints the customer nicely.
<br><br>
+ **CartItem** is a custom `data` type representing one line inside a shopping cart. It has two fields:
  + `product` of type **Product**
  + and `quantity` of type `Int`.
  Its custom `Show` instance prints each item together with the number of units.
<br><br>
+ **ShoppingCart** is a `newtype` wrapper around a list of **CartItem** values. The wrapper makes the type safer and easier to handle. The custom `Show` instance prints the whole cart item by item, using indentation for clarity.
<br><br>
+ **Stock** is a `newtype` wrapper around a list of pairs `(Product, Int)`. Each pair stores a product and the number of units available. Its `Show` instance prints the whole stock clearly, showing product info and available quantity.
<br><br>
+ **Status** is a `data` type that represents the state of an order. It has five possible values: `Pending`, `Processing`, `Shipped`, `Delivered`, and `Cancelled`. This type allows the program to model order progression.
<br><br>
+ **Order** is a custom `data` type representing a complete purchase. It contains:
  + a **Customer**,
  + a **ShoppingCart**,
  + a `totalPrice` of type `Float`,
  and a **Status**.
  Its `Show` instance prints all this information in a readable format.
<br><br>
+ **SearchCriterion** is a `data` type used to filter lists of orders. It includes several constructors: `ById`, `ByLoyaltyLevel`, `ByProductId`, `ByCategory`, and `ByTotalPrice`, each carrying the needed value.
<br><br>
+ **Discountable** is a type class that defines the function `applyDiscount`. Any type that becomes an instance of this class must explain how it transforms a price.
  Both **LoyaltyLevel** and **Category** are instances, meaning the program can apply discounts based on customer loyalty, product category, or a combination of both.

In [16]:
import System.IO  -- first load modules
import Data.Char (isDigit, toLower)
import Text.Read (readMaybe)

In [17]:
data Category = Electronics | Books | Clothing | Groceries deriving (Show, Eq, Read)

data Product = Product {pid :: Int, pname :: String, price :: Float, category :: Category}
instance Show Product where
    show (Product id name price category) = "Product id: " ++ show id ++ ", product: " ++ show name ++ ", price: " ++ show price ++ ", category: " ++ show category

In [18]:
instance Eq Product where
        (Product pid1 _ _ _) == (Product pid2 _ _ _) = pid1 == pid2 --add on from 5 - needed in 5

In [19]:
data LoyaltyLevel = Bronze | Silver | Gold deriving (Show, Eq, Read)

data Customer = Customer {cid :: Int, cname :: String, loyaltyLevel :: LoyaltyLevel}

instance Show Customer where
    show (Customer id name loyaltyLevel) = "Customer id: " ++ show id ++ ", customer: " ++ show name ++ ", loyalty level: " ++ show loyaltyLevel

instance Eq Customer where
  (Customer id1 _ _) == (Customer id2 _ _) = id1 == id2

In [20]:
data CartItem = CartItem {product :: Product, quantity :: Int}

instance Show CartItem where
    show (CartItem product quantity) = "Item: " ++ show product ++ ", units: " ++ show quantity

instance Eq CartItem where
        (CartItem p1 q1) == (CartItem p2 q2) = p1 == p2 && q1 == q2

In [21]:
newtype ShoppingCart = ShoppingCart [CartItem]

instance Show ShoppingCart where
    show (ShoppingCart items) = "Shopping cart:\n" ++ unlines (map (\x -> "\t" ++ show x) items)

instance Eq ShoppingCart where
        (ShoppingCart items1) == (ShoppingCart items2) = items1 == items2

In [23]:
newtype Stock = Stock [(Product, Int)] -- product and their available quantity

instance Show Stock where
    show (Stock items) = "Stock:\n" ++ unlines (map (\(p, i) -> "\t" ++ show p ++ ", quantity available: " ++ show i) items)

data Status = Pending | Processing | Shipped | Delivered | Cancelled deriving (Show, Read, Eq)

In [24]:
data Order = Order {customer :: Customer, shoppingCart :: ShoppingCart, totalPrice :: Float, status :: Status}

instance Show Order where
    show (Order customer shoppingCart totalPrice status) = "Order for customer: " ++ show customer ++ "\n" ++ show shoppingCart 
        ++ "Total price: " ++ show totalPrice ++ ", status: " ++ show status ++ "\n"

instance Eq Order where
    (Order (Customer cid1 _ _) shpcrt1 _ _) == (Order (Customer cid2 _ _) shpcrt2 _ _) = cid1 == cid2 && shpcrt1 == shpcrt2

data SearchCriterion = ById Int | ByLoyaltyLevel LoyaltyLevel | ByProductId Int | ByCategory Category | ByTotalPrice Float deriving Show

In [25]:
class Discountable a where
  applyDiscount :: a -> Float -> Float

instance Discountable LoyaltyLevel where
  applyDiscount Bronze p = p
  applyDiscount Silver p = p * 0.95
  applyDiscount Gold   p = p * 0.90


instance Discountable Category where
  applyDiscount Books p = p * 0.85 -- Books 15% off
  applyDiscount _     p = p

In [26]:
newtype Error = Error [Product] deriving Show --created due to 7

### Testing

We ran these small tests to verify that each of our types is correctly constructed and printed. By creating example values such as **Product**, **CartItem**, **ShoppingCart**, **Stock**, **Customer**, and **Order**, we checked that their fields are stored properly and that our custom `Show` instances display the information in a clear and readable way. This confirms that the basic data structures of the system behave as expected.

In [27]:
product1 = Product 1 "Ivan" 12 Electronics
product2 = Product 2 "Isa" 15 Clothing
product3 = Product 3 "Siro" 5 Groceries

cart1 = CartItem product1 2
cart2 = CartItem product2 5

cart = ShoppingCart [cart1,  cart2]
cart

stock = Stock (zip [product1, product2, product3] [2, 3, 4])

stock

customer = Customer 1 "Ivan" Gold
order = Order customer cart 17 Pending

order

Shopping cart:
	Item: Product id: 1, product: "Ivan", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5

Stock:
	Product id: 1, product: "Ivan", price: 12.0, category: Electronics, quantity available: 2
	Product id: 2, product: "Isa", price: 15.0, category: Clothing, quantity available: 3
	Product id: 3, product: "Siro", price: 5.0, category: Groceries, quantity available: 4

Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ivan", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
Total price: 17.0, status: Pending

3. Price Calculation

The function `calculateProductPrice` was designed to extract only the price field from a **Product**, ignoring all other information via pattern matching: we match the constructor, discard the unused fields, and return the price `p`. This keeps the function efficient.

In [28]:
calculateProductPrice :: Product -> Float
calculateProductPrice (Product _ _ p _) = p

### Testing

In [29]:
calculateProductPrice product1

12.0

4. Discount Application

`calculateOrderTotal` was designed to compute the final cost of an order by combining quantities and discounts in a clear way. For each `CartItem` in the `ShoppingCart`, we multiply the quantity by the discounted price of its `Product`. The helper `priceAfter` first applies the category discount and then the loyalty discount, using the `Discountable` type class. We use a list comprehension and `sum` to add all these subtotals, so the function directly reflects:

$$
\text{total} = \sum_{\text{items}} (\text{quantity} \times \text{discounted price})
$$

In [30]:
calculateOrderTotal :: Order -> Float
calculateOrderTotal (Order cust (ShoppingCart items) _ _) =
  sum [ fromIntegral (quantity it) * priceAfter (product it) | it <- items ]
  where
    priceAfter prod = applyDiscount (loyaltyLevel cust)(applyDiscount (category prod)(calculateProductPrice prod))

Testing

In [31]:
product1 = Product 1 "Ipad" 12 Electronics
product2 = Product 2 "Blouse"  15 Clothing
product3 = Product 3 "Lemons"  5 Groceries

cart1 = CartItem product1 2
cart2 = CartItem product2 5
cart  = ShoppingCart [cart1, cart2]

stock = Stock (zip [product1, product2, product3] [2, 3, 4])

customerGold   = Customer 1 "Ivan"   Gold
customerSilver = Customer 2 "Isa"  Silver
customerBronze = Customer 3 "Siro"  Bronze

orderGold   = Order customerGold   cart 0 Pending
orderSilver = Order customerSilver cart 0 Pending
orderBronze = Order customerBronze cart 0 Pending

calculateOrderTotal orderGold
calculateOrderTotal orderSilver
calculateOrderTotal orderBronze

89.1

94.05

99.0

In [32]:
productBook = Product 4 "Functional Programming for Dummies 101" 40 Books
cartBook1   = CartItem productBook 1
cartBook2   = CartItem productBook 2

cartwBook1 = ShoppingCart [cartBook1]
cartwBook2 = ShoppingCart [cartBook2]

orderBookGold1   = Order customerGold   cartwBook1 0 Pending
orderBookSilver1 = Order customerSilver cartwBook1 0 Pending
orderBookBronze1 = Order customerBronze cartwBook1 0 Pending

calculateOrderTotal orderBookGold1
calculateOrderTotal orderBookSilver1
calculateOrderTotal orderBookBronze1

30.599998

32.3

34.0

In [33]:
-- original cart + 2 books
cartMixed = ShoppingCart [cart1, cart2, cartBook2]
orderMixedGold = Order customerGold cartMixed 0 Pending

calculateOrderTotal orderMixedGold

150.29999

5. Purchase

`addToCart` was designed to update a shopping cart in a simple and safe way. If the cart is empty, it just creates a new cart with the given `CartItem`. If not, it goes through the list of items with pattern matching: when it finds the same `Product` (using the `Eq` instance for **Product**), it increases its quantity; otherwise, it keeps the current item and continues recursively with the rest of the cart. The helper `add` is used to rebuild the cart when the product is not at the head, so we keep all existing items and only change the one that matches.

In [34]:
add :: CartItem -> ShoppingCart -> ShoppingCart
add item (ShoppingCart xs) = ShoppingCart (item:xs)

In [35]:
addToCart :: CartItem -> ShoppingCart -> ShoppingCart
addToCart item (ShoppingCart []) = ShoppingCart [item]
addToCart i@(CartItem product quantity) (ShoppingCart ((CartItem p q): cartList)) 
                | product == p = ShoppingCart (CartItem p (q+quantity) : cartList)
                | otherwise = add (CartItem p q) (addToCart i (ShoppingCart cartList))

In [36]:
add cart1 cart
cart
addToCart cart1 cart
addToCart (CartItem product3 12) cart

Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.0, category: Electronics, units: 2
	Item: Product id: 1, product: "Ipad", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.0, category: Clothing, units: 5

Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.0, category: Clothing, units: 5

Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.0, category: Electronics, units: 4
	Item: Product id: 2, product: "Blouse", price: 15.0, category: Clothing, units: 5

Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.0, category: Clothing, units: 5
	Item: Product id: 3, product: "Lemons", price: 5.0, category: Groceries, units: 12

6. Cart Validation

`stockQuantity` was designed to search the stock list for a specific **Product** and return its available quantity. It uses list comprehension to collect all matching quantities, and if none are found it returns `0`.

`checkStock` uses `stockQuantity` to validate the cart. It scans all `CartItem` values in the **ShoppingCart**, and collects those products for which the available stock is smaller than the quantity requested. This gives a direct and readable way to detect missing or insufficient items before creating an order.

In [37]:
stockQuantity :: Stock -> Product -> Int
stockQuantity (Stock xs) p
  | null matches = 0
  | otherwise    = head matches
  where
    matches = [q | (r, q) <- xs, r == p]

checkStock :: Stock -> ShoppingCart -> [Product]
checkStock stock (ShoppingCart items) =
  [ p | CartItem p q <- items, stockQuantity stock p < q ]

TESTS

In [39]:
-- stock = Stock (zip [product1, product2, product3] [2, 3, 4])
--cartOK = ShoppingCart [ CartItem product1 2, CartItem product2 1]
--checkStock cartOK

In [40]:
--cartOver = ShoppingCart[ CartItem product1 2, CartItem product2 5]
--checkStock cartOver

In [41]:
--cartMultiple = ShoppingCart[ CartItem product1 3, CartItem product2 2, CartItem product3 10]
--checkStock cartMultiple

In [43]:
--cartEmpty = ShoppingCart []
--checkStock cartEmpty

7. Order Creation

`createOrder` was designed to validate a cart before building an **Order**. It first checks which products are missing using `checkStock`. If the list is empty, stock is sufficient, so the function returns `Right` with a new order in the **Pending** state and its total price computed by `calculateOrderTotal`. If any product is missing, it returns `Left (Error missing)`. An order can only be created when all items are available.

In [44]:
--createOrder :: Customer -> ShoppingCart -> Either Error Order
--createOrder customer c@(ShoppingCart xs) 
--        | null missing = Right (Order customer c totalPrice Pending)
--        | otherwise = Left (Error missing)
--        where 
--                missing = checkStock c
--                totalPrice = calculateOrderTotal (Order customer c 0 Pending)
                
createOrder :: Stock -> Customer -> ShoppingCart -> Either Error Order
createOrder stock customer c@(ShoppingCart xs)
  | null missing = Right (Order customer c totalPrice Pending)
  | otherwise    = Left (Error missing)
  where
    missing    = checkStock stock c
    totalPrice = calculateOrderTotal (Order customer c 0 Pending)


In [45]:
--createOrder customerGold cartMultiple
--createOrder customerGold cartOK

8. Status update

`transitionAllowed` defines all the valid movements between order **Status** values. Each allowed pair is listed explicitly, and any other combination returns `False`. We explicitly list the transicions since succ and pred wouldn't consider imposible transitions.

`updateOrderStatus` uses this rule to control state changes. It compares the old status of the order with the new one: if the transition is valid, it returns `Just` with the updated order; if not, it returns `Nothing`. This ensures that orders can only move forward through logical states and prevents impossible transitions.

In [46]:
transitionAllowed :: Status -> Status -> Bool
transitionAllowed Pending Processing = True
transitionAllowed Pending Cancelled  = True
transitionAllowed Processing Shipped    = True
transitionAllowed Processing Cancelled  = True
transitionAllowed Shipped Delivered  = True
transitionAllowed Shipped Cancelled  = True
transitionAllowed _ _ = False

updateOrderStatus :: Order -> Status -> Maybe Order
updateOrderStatus o@(Order _ _ _ old) new
  | transitionAllowed old new = Just (o { status = new })
  | otherwise             = Nothing

TESTS

In [89]:
orderTest = Order customer cart 0 Pending

-- Valid transitions
updateOrderStatus orderTest Processing
updateOrderStatus orderTest Cancelled   
-- Invalid transitions
orderDelivered = Order customer cart 0 Delivered
updateOrderStatus orderDelivered Pending
updateOrderStatus orderDelivered Shipped

orderProc  = updateOrderStatus orderTest Processing
orderShip  = orderProc  >>= (`updateOrderStatus` Shipped)
orderDeliv = orderShip  >>= (`updateOrderStatus` Delivered)

orderProc
orderShip
orderDeliv

Just Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.0, category: Clothing, units: 5
Total price: 0.0, status: Processing

Just Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.0, category: Clothing, units: 5
Total price: 0.0, status: Cancelled

Nothing

Nothing

Just Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.0, category: Clothing, units: 5
Total price: 0.0, status: Processing

Just Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.0, category: Clothing, units: 5
Total price: 0.0, status: Shipped

Just Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.0, category: Clothing, units: 5
Total price: 0.0, status: Delivered

9. Polymorphic Search:

In [47]:
isProductByIdInCart :: Int -> ShoppingCart -> Bool
isProductByIdInCart _ (ShoppingCart []) = False
isProductByIdInCart productId (ShoppingCart ((CartItem (Product pid _ _ _) _):xs)) 
        | productId == pid = True
        | otherwise = isProductByIdInCart productId (ShoppingCart xs)


isProductByCategoryInCart :: Category -> ShoppingCart -> Bool
isProductByCategoryInCart _ (ShoppingCart []) = False
isProductByCategoryInCart category (ShoppingCart ((CartItem (Product _ _ _ c) _):xs)) 
        | category == c = True
        | otherwise = isProductByCategoryInCart category (ShoppingCart xs)



searchOrders :: [SearchCriterion] -> [Order] -> [Order]
searchOrders [] orders = orders
searchOrders ((ById id): xs) orders = searchOrders xs (filter (\(Order (Customer cid _ _) _ _ _) -> cid == id) orders)
searchOrders ((ByLoyaltyLevel loyaltyLevel): xs) orders = searchOrders xs (filter (\(Order (Customer _ _ ll) _ _ _) -> loyaltyLevel == ll) orders)
searchOrders ((ByProductId id): xs) orders = searchOrders xs (filter (\(Order _ cart _ _) -> isProductByIdInCart id cart) orders)
searchOrders ((ByCategory category): xs) orders = searchOrders xs (filter (\(Order _ cart _ _) -> isProductByCategoryInCart category cart) orders)
searchOrders ((ByTotalPrice price): xs) orders = searchOrders xs (filter (\(Order _ _ p _) -> price == p) orders)

In [91]:
product1 = Product 1 "Ivan" 12 Electronics
product2 = Product 2 "Isa" 15 Clothing
product3 = Product 3 "Siro" 5 Groceries

item1 = CartItem product1 2
item2 = CartItem product2 5
item3 = CartItem product3 10

cart1 = ShoppingCart [item1,  item2]
cart2 = ShoppingCart [item2,  item3]


customer1 = Customer 1 "Ivan" Gold
customer2 = Customer 2 "Isa" Bronze
order1 = Order customer1 cart1 17 Pending
order2 = Order customer2 cart2 20 Processing
order3 = Order customer2 cart2 17 Shipped


searchOrders [ById 2] [order1, order2, order3]
searchOrders [ByTotalPrice 17] [order1, order2, order3]
searchOrders [ByTotalPrice 17, ById 2] [order1, order2, order3]

[Order for customer: Customer id: 2, customer: "Isa", loyalty level: Bronze
Shopping cart:
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
	Item: Product id: 3, product: "Siro", price: 5.0, category: Groceries, units: 10
Total price: 20.0, status: Processing
,Order for customer: Customer id: 2, customer: "Isa", loyalty level: Bronze
Shopping cart:
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
	Item: Product id: 3, product: "Siro", price: 5.0, category: Groceries, units: 10
Total price: 17.0, status: Shipped
]

[Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ivan", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
Total price: 17.0, status: Pending
,Order for customer: Customer id: 2, customer: "Isa", loyalty level: Bronze
Shopping cart:
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
	Item: Product id: 3, product: "Siro", price: 5.0, category: Groceries, units: 10
Total price: 17.0, status: Shipped
]

[Order for customer: Customer id: 2, customer: "Isa", loyalty level: Bronze
Shopping cart:
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
	Item: Product id: 3, product: "Siro", price: 5.0, category: Groceries, units: 10
Total price: 17.0, status: Shipped
]

10. Customer Management

In [50]:
-- helper 1
isActive :: Status -> Bool
isActive Delivered = False
isActive Cancelled = False
isActive _         = True

-- helper 2
activeOrders :: [Order] -> [Order]
activeOrders os =
  [ o | o@(Order _ _ _ st) <- os, isActive st ] --call to helper1
-- helper 3
customersOf :: [Order] -> [Customer]
customersOf os =
  [ c | Order c _ _ _ <- os ]

-- helper4 (no duplicates)
uniq :: Eq a => [a] -> [a]
uniq [] = []
uniq (x:xs)
  | x `elem` xs = uniq xs
  | otherwise   = x : uniq xs

-- helper 5
totalOfCustomer :: Customer -> [Order] -> Float
totalOfCustomer cust os =
  sum [ t | Order c _ t _ <- os, c == cust ]

In [51]:
highValueCustomers :: [Order] -> Float -> [Customer]
highValueCustomers orders limit =
  [ c | c <- uniq (customersOf active), totalOfCustomer c active > limit] -- calls to helper 3 and helper 5
  where
    active = activeOrders orders --call ti helper 2

In [94]:
customer1 = Customer 1 "Ivan" Gold
customer2 = Customer 2 "Isa" Gold
customer3 = Customer 3 "Siro" Bronze
customer4 = Customer 4 "Yan" Silver

product1 = Product 1 "Book" 12 Books
product2 = Product 2 "Lemons" 1.5 Groceries
product3 = Product 3 "Sandals" 50 Clothing
product4 = Product 4 "IPhone" 999 Electronics

item1 = CartItem product1 2
item2 = CartItem product1 1

item3 = CartItem product2 5
item4 = CartItem product2 2

item5 = CartItem product3 2
item6 = CartItem product3 1

item7 = CartItem product4 1
item8 = CartItem product4 2

cart1 = ShoppingCart [item1, item3]
cart2 = ShoppingCart [item2, item4]
cart3 = ShoppingCart [item5, item7]
cart4 = ShoppingCart [item6, item8]
cart5 = ShoppingCart [item1, item3, item5, item7]
cart6 = ShoppingCart [item2, item4, item6, item8]
cart7 = ShoppingCart [item1, item2, item3, item4, item5, item6, item7, item8]

order1' = Order customer1 cart1 0 Pending
order1 = order1' {totalPrice = calculateOrderTotal order1'}
order2' = Order customer1 cart2 0 Processing
order2 = order2' {totalPrice = calculateOrderTotal order2'}
order3' = Order customer1 cart3 0 Delivered
order3 = order3' {totalPrice = calculateOrderTotal order3'}

order4' = Order customer2 cart4 0 Shipped
order4 = order4' {totalPrice = calculateOrderTotal order4'}

order5' = Order customer3 cart5 0 Cancelled
order5 = order5' {totalPrice = calculateOrderTotal order5'}
order6' = Order customer3 cart6 0 Shipped
order6 = order6' {totalPrice = calculateOrderTotal order6'}

order7' = Order customer4 cart7 0 Cancelled
order7 = order7' {totalPrice = calculateOrderTotal order7'}

orders = [order1, order2, order3, order4, order5, order6, order7]
unique = uniq $ customersOf orders

active = activeOrders orders

-- totalOfCustomer customer2 active
highValueCustomers orders 1900

[Customer id: 3, customer: "Siro", loyalty level: Bronze]

11. Functions to use I/O to allow the user to interact with the system. He/she will be able to search by Product, by Category or by product maximum price, to add elements to the Cart and to place an Order.

In [52]:
-- (REUSED from 12 slightly improved :)
splitOn :: Char -> String -> [String]
splitOn _ "" = [""]
splitOn delimiter (x:xs)
    | x == delimiter = "" : rest
    | otherwise      = (x : head rest) : tail rest
  where
    rest = splitOn delimiter xs

listToString :: Show a => [a] -> String
listToString []     = ""
listToString (x:xs) = show x ++ "\n" ++ listToString xs
--------------------------------------------------------------------------------
-- File I/O: Customers  (recycling Ivan's wonderful logic)
--------------------------------------------------------------------------------
parseCustomer :: Customer -> String
parseCustomer (Customer cid name ll) =
  show cid ++ "," ++ name ++ "," ++ show ll

saveCustomers :: Handle -> [Customer] -> IO ()
saveCustomers h = mapM_ (hPutStrLn h . parseCustomer)

parseCustomerLine :: String -> Customer
parseCustomerLine line = Customer cid name ll
  where
    [cidStr, name, llStr] = splitOn ',' line
    cid = read cidStr
    ll  = read llStr

readCustomer :: FilePath -> IO [Customer]
readCustomer fp = do
  contents <- readFile fp
  return (map parseCustomerLine (lines contents))
--------------------------------------------------------------------------------
-- File I/O: Stock  (using Ivan's logic)
--------------------------------------------------------------------------------
parseStock :: Stock -> String
parseStock (Stock items) =
  unlines
    [ show pid ++ "," ++ name ++ "," ++ show price ++ "," ++ show cat ++ "," ++ show qty | (Product pid name price cat, qty) <- items ]

saveStock :: Handle -> Stock -> IO ()
saveStock h stock = hPutStrLn h (parseStock stock)

parseProductQuantityLine :: String -> (Product, Int)
parseProductQuantityLine line = (Product pid name price cat, qty)
  where
    [pidStr, name, priceStr, catStr, qtyStr] = splitOn ',' line
    pid   = read pidStr
    price = read priceStr
    cat   = read catStr
    qty   = read qtyStr

readStock :: FilePath -> IO Stock
readStock fp = do
  contents <- readFile fp
  let stockItems = map parseProductQuantityLine (filter (not . null) (lines contents))
  return (Stock stockItems)
--------------------------------------------------------------------------------
-- File I/O: Orders  (Ivan's)
--------------------------------------------------------------------------------
parsePriceStatusLine :: String -> (Float, Status)
parsePriceStatusLine line = (totalPrice, status)
  where
    [priceStr, statusStr] = splitOn ',' (drop 6 line)
    totalPrice = read priceStr
    status     = read statusStr

readCartItems :: [String] -> ([CartItem], [String]) -- Read cart item lines until "Price"
readCartItems [] = ([], [])
readCartItems (line:rest)
  | take 5 line == "Price" = ([], line:rest)
  | otherwise              = (CartItem product qty : items, remaining)
  where
    (items, remaining) = readCartItems rest
    (product, qty)     = parseProductQuantityLine line

readOrderFromLines :: [String] -> (Order, [String])
readOrderFromLines [] = error "No order data"
readOrderFromLines (customerLine:rest) = (order, remaining)
  where
    customer = parseCustomerLine (drop 9 customerLine)  -- drop "Customer "
    (cartItems, priceAndRest) = readCartItems rest
    (totalPrice, status)      = parsePriceStatusLine (head priceAndRest)
    shoppingCart              = ShoppingCart cartItems
    order                     = Order customer shoppingCart totalPrice status
    remaining                 = tail priceAndRest

readAllOrdersFromLines :: [String] -> [Order] --removed let/in
readAllOrdersFromLines [] = []
readAllOrdersFromLines ls =
  case readOrderFromLines ls of
    (order, remaining) -> order : readAllOrdersFromLines remaining

readOrdersFromFile :: FilePath -> IO [Order]
readOrdersFromFile fp = do
  contents <- readFile fp
  return (readAllOrdersFromLines (lines contents))

-- Writing orders
formatCartItem :: CartItem -> String
formatCartItem (CartItem (Product pid name price cat) qty) =
  show pid ++ "," ++ name ++ "," ++ show price ++ "," ++ show cat ++ "," ++ show qty

formatPriceStatus :: Float -> Status -> String
formatPriceStatus totalPrice status =
  "Price " ++ show totalPrice ++ "," ++ show status

writeOrderToHandle :: Handle -> Order -> IO ()
writeOrderToHandle h (Order cust (ShoppingCart items) totalPrice status) = do
  hPutStrLn h ("Customer " ++ parseCustomer cust)
  mapM_ (hPutStrLn h . formatCartItem) items
  hPutStrLn h (formatPriceStatus totalPrice status)

writeOrdersToFile :: FilePath -> [Order] -> IO ()
writeOrdersToFile fp orders =
  withFile fp WriteMode $ \h ->
    mapM_ (writeOrderToHandle h) orders

In [54]:
 -- each user needs to identify himself/herself --helper 1 - search by name
searchProductsByName :: String -> [Product] -> [Product]
searchProductsByName name ps =
  [ p | p@(Product _ n _ _) <- ps, n == name ]
  
--helper 2- search by category
searchProductsByCategory :: Category -> [Product] -> [Product]
searchProductsByCategory cat ps =
  [ p | p@(Product _ _ _ c) <- ps, c == cat ]

-- helper 3 - search by max price
searchProductsByMaxPrice :: Float -> [Product] -> [Product]
searchProductsByMaxPrice maxP ps =
  [ prod | prod@(Product _ _ price _) <- ps, price <= maxP ]
  
-- helper 4- prod by id
findProductById :: Int -> [Product] -> Maybe Product
findProductById _ [] = Nothing
findProductById pid (p@(Product pid' _ _ _) : ps)
  | pid == pid' = Just p
  | otherwise   = findProductById pid ps

-- helper 5 - read categ from user
readCategory :: String -> Maybe Category
readCategory "Electronics" = Just Electronics
readCategory "Books"       = Just Books
readCategory "Clothing"    = Just Clothing
readCategory "Groceries"   = Just Groceries
readCategory _             = Nothing

-- helper 6 - printer
printProducts :: [Product] -> IO ()
printProducts [] = putStrLn "No products found."
printProducts [p] = print p
printProducts (p:ps) = do
  print p
  printProducts ps

In [55]:
--------------------------------------------------------------------------------
-- Product search I/O (use Stock -> derive catalog)
--------------------------------------------------------------------------------
catalogFromStock :: Stock -> [Product]
catalogFromStock (Stock items) = [ p | (p, qty) <- items, qty > 0 ]

searchProductIO :: Stock -> IO ()
searchProductIO stock = do
  let catalog = catalogFromStock stock
  putStrLn "Enter product name:"
  name <- getLine
  printProducts (searchProductsByName name catalog)

searchCategoryIO :: Stock -> IO ()
searchCategoryIO stock = do
  let catalog = catalogFromStock stock
  putStrLn "Enter category (Electronics, Books, Clothing, Groceries):"
  s <- getLine
  case readCategory s of
    Nothing  -> putStrLn "Unknown category."
    Just cat -> printProducts (searchProductsByCategory cat catalog)

searchMaxPriceIO :: Stock -> IO ()
searchMaxPriceIO stock = do
  let catalog = catalogFromStock stock
  putStrLn "Enter maximum price:"
  s <- getLine
  case readMaybe s :: Maybe Float of
    Nothing  -> putStrLn "That is not a number."
    Just mp  -> printProducts (searchProductsByMaxPrice mp catalog)

In [56]:
--------------------------------------------------------------------------------
-- Add to cart I/O (uses addToCart from 5)
--------------------------------------------------------------------------------
addToCartIO :: Stock -> ShoppingCart -> IO ShoppingCart
addToCartIO stock cart = do
  let catalog = catalogFromStock stock
  putStrLn "Enter product ID:"
  sId <- getLine
  putStrLn "Enter quantity:"
  sQty <- getLine
  case (readMaybe sId :: Maybe Int, readMaybe sQty :: Maybe Int) of
    (Just pid, Just qty) -> addToCartHelper pid qty catalog cart
    _   -> do
      putStrLn "Invalid ID or quantity."
      return cart

addToCartHelper :: Int -> Int -> [Product] -> ShoppingCart -> IO ShoppingCart
addToCartHelper pid qty catalog cart =
  addToCartMaybe (findProductById pid catalog) qty cart

addToCartMaybe :: Maybe Product -> Int -> ShoppingCart -> IO ShoppingCart
addToCartMaybe Nothing _ cart = do
  putStrLn "Product not found."
  return cart
addToCartMaybe (Just p) qty cart = do
  putStrLn "Product added to cart."
  return (addToCart (CartItem p qty) cart)

Line 18: Eta reduce
Found:
addToCartHelper pid qty catalog cart
  = addToCartMaybe (findProductById pid catalog) qty cart
Why not:
addToCartHelper pid qty catalog
  = addToCartMaybe (findProductById pid catalog) qty

In [58]:
--------------------------------------------------------------------------------
-- Customer identification (new vs existing)
--------------------------------------------------------------------------------
findCustomerById :: Int -> [Customer] -> Maybe Customer
findCustomerById _ [] = Nothing
findCustomerById cid (c@(Customer cid' _ _) : cs)
  | cid == cid' = Just c
  | otherwise   = findCustomerById cid cs

readLoyaltyLevel :: String -> Maybe LoyaltyLevel
readLoyaltyLevel s =
  case map toLower s of
    "bronze" -> Just Bronze
    "silver" -> Just Silver
    "gold" -> Just Gold
    _  -> Nothing

nextCustomerId :: [Customer] -> Int
nextCustomerId [] = 1
nextCustomerId cs = 1 + maximum [cid | Customer cid _ _ <- cs]

identifyCustomer :: [Customer] -> IO (Customer, [Customer])
identifyCustomer customers = do
  putStrLn "Are you an existing customer (1) or a new customer (2)? (b to exit)"
  ans <- getLine
  case ans of
    "1" -> existingCustomerFlow customers
    "2" -> newCustomerFlow customers
    "b" -> do
      putStrLn "Goodbye."
      fail "User aborted"
    _   -> do
      putStrLn "I did not understand that."
      identifyCustomer customers

existingCustomerFlow :: [Customer] -> IO (Customer, [Customer])
existingCustomerFlow customers = do
  putStrLn "Please enter your customer ID:"
  sId <- getLine
  case readMaybe sId :: Maybe Int of
    Nothing -> do
      putStrLn "ID must be an integer."
      existingCustomerFlow customers
    Just cid ->
      case findCustomerById cid customers of
        Nothing -> do
          putStrLn "No customer with that ID."
          existingCustomerFlow customers
        Just c -> do
          putStrLn ("Welcome back, " ++ cname c ++ "!")
          return (c, customers)

newCustomerFlow :: [Customer] -> IO (Customer, [Customer])
newCustomerFlow customers = do
  putStrLn "Registering a new customer."
  putStrLn "Enter your name:"
  name <- getLine
  putStrLn "Enter your loyalty level (Bronze, Silver, Gold):"
  llStr <- getLine
  maybe (retry customers) (success customers name) (readLoyaltyLevel llStr)

retry :: [Customer] -> IO (Customer, [Customer])
retry customers = do
  putStrLn "Unknown loyalty level."
  newCustomerFlow customers

success :: [Customer] -> String -> LoyaltyLevel -> IO (Customer, [Customer])
success customers name ll = do
  putStrLn ("Your new customer ID is " ++ show cid)
  return (newC, customers ++ [newC])
  where
    cid  = nextCustomerId customers
    newC = Customer cid name ll

Stock update after succesful order

In [59]:
updateStockWithCart :: Stock -> ShoppingCart -> Stock
updateStockWithCart (Stock items) (ShoppingCart cartItems) =
  Stock (foldl updateOne items cartItems)
  where
    updateOne :: [(Product, Int)] -> CartItem -> [(Product, Int)]
    updateOne acc (CartItem (Product pid _ _ _) q) =
      [ if pid' == pid then (p, qty - q) else (p, qty) | (p@(Product pid' _ _ _), qty) <- acc ]

Shopping loop

In [60]:
shopLoop :: Stock -> ShoppingCart -> IO (ShoppingCart, Stock)
shopLoop stock cart = do
  let catalog = catalogFromStock stock
  putStrLn "\nWhat would you like to do?"
  putStrLn "1. Search products by name"
  putStrLn "2. Search products by category"
  putStrLn "3. Search products by maximum price"
  putStrLn "4. Add product to cart"
  putStrLn "5. View cart"
  putStrLn "6. Checkout"
  putStrLn "b. Exit without ordering"
  choice <- getLine
  case choice of
    "1" -> do
      searchProductIO stock
      shopLoop stock cart
    "2" -> do
      searchCategoryIO stock
      shopLoop stock cart
    "3" -> do
      searchMaxPriceIO stock
      shopLoop stock cart
    "4" -> do
      cart' <- addToCartIO stock cart
      shopLoop stock cart'
    "5" -> do
      putStrLn "Your cart:"
      print cart
      shopLoop stock cart
    "6" -> return (cart, stock)
    "b" -> return (cart, stock)
    _   -> do
      putStrLn "Unknown option."
      shopLoop stock cart

In [61]:
--------------------------------------------------------------------------------
-- Place order using createOrder (from 7) and update files
--------------------------------------------------------------------------------
placeOrderIO :: Stock -> Customer -> ShoppingCart -> IO (Either Error Order)
placeOrderIO stock cust cart =
  case createOrder stock cust cart of
    Left err -> do
      putStrLn ("Order could not be created: " ++ show err)
      return (Left err)
    Right order -> do
      putStrLn "Order created successfully:"
      print order
      return (Right order)

customerSession :: IO ()
customerSession = do
  putStrLn "Welcome to the shop!"

  customers <- readCustomer "customers.txt"
  stock     <- readStock    "stock.txt"
  orders    <- readOrdersFromFile "orders.txt"

  (customer, customers') <- identifyCustomer customers

  let emptyCart = ShoppingCart []

  (finalCart, stockBeforeOrder) <- shopLoop stock emptyCart

  case finalCart of
    ShoppingCart [] -> do
      putStrLn "Your cart is empty. Nothing to do."
      withFile "customers.txt" WriteMode $ \h -> saveCustomers h customers'
      withFile "stock.txt"     WriteMode $ \h -> saveStock h stockBeforeOrder
      writeOrdersToFile "orders.txt" orders
      putStrLn "Goodbye."
    _ -> do
      result <- placeOrderIO stockBeforeOrder customer finalCart
      case result of
        Left _ -> do
          putStrLn "Order failed. No changes saved (except new customers)."
          withFile "customers.txt" WriteMode $ \h -> saveCustomers h customers'
          withFile "stock.txt"     WriteMode $ \h -> saveStock h stockBeforeOrder
          writeOrdersToFile "orders.txt" orders
        Right newOrder -> do
          let newStock  = updateStockWithCart stockBeforeOrder finalCart
              newOrders = orders ++ [newOrder]
          putStrLn "Updating files..."
          withFile "customers.txt" WriteMode $ \h -> saveCustomers h customers'
          withFile "stock.txt"     WriteMode $ \h -> saveStock h newStock
          writeOrdersToFile "orders.txt" newOrders
          putStrLn "Order placed and data saved. Thank you!"

main :: IO ()
main = customerSession

12. Functions to use I/O to manage the shop. The shop owner will be able to search by user ID, by Loyalty Level or by high value customers, and to process and ship all the pending orders

In [102]:
-- Function to split a string by a delimiter
splitOn :: Char -> String -> [String]
splitOn _ "" = [""]
splitOn delimiter (x:xs)
    | x == delimiter = "" : rest
    | otherwise = (x : head rest) : tail rest
  where
    rest = splitOn delimiter xs


listToString :: Show a => [a] -> String
listToString [] = ""
listToString (x:xs) = show x ++ "\n" ++ listToString xs


### Interaction with files

Functions to save and read Customer to/from file

In [103]:
parseCustomer :: Customer -> String
parseCustomer (Customer id name loyaltyLevel) = show id ++ "," ++ name ++ "," ++ show loyaltyLevel

saveCustomers :: Handle -> [Customer] -> IO ()
saveCustomers handle = mapM_ (hPutStrLn handle . parseCustomer)

parseCustomerLine :: String -> Customer
parseCustomerLine line = Customer id name loyaltyLevel
  where
    [idStr, name, loyaltyLevelStr] = splitOn ',' line
    id = read idStr :: Int
    loyaltyLevel = read loyaltyLevelStr :: LoyaltyLevel

readCustomer :: FilePath -> IO [Customer]
readCustomer filepath = do
    contents <- readFile filepath
    return (map parseCustomerLine (lines contents))

In [104]:
main :: IO ()
main = do    
    -- Save customers
    let customers = [Customer 1 "Alice" Gold, Customer 2 "Bob" Silver]
    withFile "customers.txt" WriteMode $ \handle -> 
        saveCustomers handle customers
    putStrLn "Customers saved!"
    
    -- Load products
    loadedCustomers <- readCustomer "customers.txt"
    putStrLn "Customers loaded:"
    putStrLn $ listToString loadedCustomers
main

Customers saved!
Customers loaded:
Customer id: 1, customer: "Alice", loyalty level: Gold
Customer id: 2, customer: "Bob", loyalty level: Silver

Functions to save and read Stock to/from file

In [111]:
parseStock :: Stock -> String
parseStock (Stock items) = unlines [show id ++ "," ++ name ++ "," ++ show price ++ "," ++ show category ++ "," ++ show quantity | (Product id name price category, quantity) <- items]

saveStock :: Handle -> Stock -> IO ()
saveStock handle stock = hPutStrLn handle (parseStock stock)

parseProductQuantityLine :: String -> (Product, Int)
parseProductQuantityLine line = (Product id name price category, qty)
  where
    [idStr, name, priceStr, categoryStr, qtyStr] = splitOn ',' line
    id = read idStr :: Int
    price = read priceStr :: Float
    category = read categoryStr :: Category
    qty = read qtyStr :: Int

readStock :: FilePath -> IO Stock
readStock filepath = do
    contents <- readFile filepath
    let stockItems = map parseProductQuantityLine (filter (not . null) (lines contents))
    return (Stock stockItems)

In [112]:
main :: IO ()
main = do
    let product1 = Product 1 "Laptop" 999.99 Electronics
    let product2 = Product 2 "Book" 19.99 Books
    let stock = Stock [(product1, 5), (product2, 10)]
    
    -- Save stock
    putStrLn "Saving stock..."
    withFile "stock.txt" WriteMode $ \handle -> 
        saveStock handle stock
    putStrLn "Stock saved!"
    
    -- Load stock
    loadedStock <- readStock "stock.txt"
    putStrLn "Stock loaded:"
    print loadedStock
    
main

Saving stock...
Stock saved!
Stock loaded:
Stock:
	Product id: 1, product: "Laptop", price: 999.99, category: Electronics, quantity available: 5
	Product id: 2, product: "Book", price: 19.99, category: Books, quantity available: 10

Functions to save and read Orders to/from file

In [113]:
-- Parse price line: "Price 17.0,Pending"
parsePriceStatusLine :: String -> (Float, Status)
parsePriceStatusLine line = (totalPrice, status)
  where
    [priceStr, statusStr] = splitOn ',' (drop 6 line)  -- Remove "Price " prefix
    totalPrice = read priceStr :: Float
    status = read statusStr :: Status


-- Read all cart item lines until we hit a "Price" line
readCartItems :: [String] -> ([CartItem], [String])
readCartItems [] = ([], [])
readCartItems (line:rest)
    | take 5 line == "Price" = ([], line:rest)
    | otherwise = (CartItem product qty : items, remaining)
  where
    (items, remaining) = readCartItems rest
    (product, qty) = parseProductQuantityLine line


-- Read a single order from a list of lines
-- Returns: (Order, remaining lines)
readOrderFromLines :: [String] -> (Order, [String])
readOrderFromLines [] = error "No order data"
readOrderFromLines (customerLine:rest) = (order, remaining)
  where
    customer = parseCustomerLine (drop 9 customerLine) -- Remove "Customer " prefix
    (cartItems, priceAndRest) = readCartItems rest
    (totalPrice, status) = parsePriceStatusLine (head priceAndRest)
    shoppingCart = ShoppingCart cartItems
    order = Order customer shoppingCart totalPrice status
    remaining = tail priceAndRest

-- Read all orders from file
readAllOrdersFromLines :: [String] -> [Order]
readAllOrdersFromLines [] = []
readAllOrdersFromLines lines = order : readAllOrdersFromLines remaining
  where
    (order, remaining) = readOrderFromLines lines

-- Read orders from file
readOrdersFromFile :: FilePath -> IO [Order]
readOrdersFromFile filepath = do
    contents <- readFile filepath
    let fileLines = lines contents
    return (readAllOrdersFromLines fileLines)

-- Format cart item for file: "1,charger,12.0,Electronics,2"
formatCartItem :: CartItem -> String
formatCartItem (CartItem (Product id name price category) qty) =
    show id ++ "," ++ name ++ "," ++ show price ++ "," ++ show category ++ "," ++ show qty

-- Format price line for file: "Price 17.0,Pending"
formatPriceStatus :: Float -> Status -> String
formatPriceStatus totalPrice status = 
    "Price " ++ show totalPrice ++ "," ++ show status

-- Write a single order to handle
writeOrderToHandle :: Handle -> Order -> IO ()
writeOrderToHandle handle (Order cust (ShoppingCart items) totalPrice status) = do
    hPutStrLn handle ("Customer " ++ parseCustomer cust)
    mapM_ (hPutStrLn handle . formatCartItem) items
    hPutStrLn handle (formatPriceStatus totalPrice status)

-- Write multiple orders to file
writeOrdersToFile :: FilePath -> [Order] -> IO ()
writeOrdersToFile filepath orders = 
    withFile filepath WriteMode $ \handle ->
        mapM_ (writeOrderToHandle handle) orders


In [114]:
-- Main function to test
main :: IO ()
main = do
    -- Create test data
    let product1 = Product 1 "charger" 12.0 Electronics
    let product2 = Product 2 "shoes" 15.0 Clothing
    let cart1 = CartItem product1 2
    let cart2 = CartItem product2 5
    let customer1 = Customer 1 "Ivan" Gold
    let order1 = Order customer1 (ShoppingCart [cart1, cart2]) 17.0 Pending
    
    let product3 = Product 3 "book" 20.0 Books
    let cart3 = CartItem product3 1
    let customer2 = Customer 2 "Maria" Silver
    let order2 = Order customer2 (ShoppingCart [cart3]) 20.0 Processing
    
    -- Save orders
    putStrLn "Saving orders..."
    writeOrdersToFile "orders.txt" [order1, order2]
    putStrLn "Orders saved!"
    
    -- Read orders
    putStrLn "\nReading orders..."
    loadedOrders <- readOrdersFromFile "orders.txt"
    putStrLn "Orders loaded:"
    mapM_ print loadedOrders

main

Saving orders...
Orders saved!

Reading orders...
Orders loaded:
Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "charger", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "shoes", price: 15.0, category: Clothing, units: 5
Total price: 17.0, status: Pending

Order for customer: Customer id: 2, customer: "Maria", loyalty level: Silver
Shopping cart:
	Item: Product id: 3, product: "book", price: 20.0, category: Books, units: 1
Total price: 20.0, status: Processing

### IO

Update status of orders of `orders` that are in `toUpdate` 

In [115]:
updateOrders :: [Order] -> [Order] -> Status -> [Order]
updateOrders toUpdate orders newStatus = map update orders
        where
            update order
                | order `elem` toUpdate  = case updateOrderStatus order newStatus of
                                        Just updatedOrder -> updatedOrder
                                        Nothing           -> order
                | otherwise = order

Line 5: Replace case with fromMaybe
Found:
case updateOrderStatus order newStatus of
  Just updatedOrder -> updatedOrder
  Nothing -> order
Why not:
Data.Maybe.fromMaybe order (updateOrderStatus order newStatus)

print orders of user with `id`, and process and ship all the pending orders

In [116]:
ordersById :: Int -> [Order] -> IO [Order]
ordersById id currentAllOrders
        | null userOrders = do
                putStrLn "No orders by user"
                searchById currentAllOrders -- let user search again
        | otherwise = do
                        putStrLn "Orders:"
                        putStrLn $ listToString userOrders
                        newAllOrders <- return $ updateOrders userOrders currentAllOrders Processing
                        newAllOrders <- return $ updateOrders userOrders newAllOrders Shipped
                        putStrLn "Orders updated"
                        start newAllOrders -- go back to start with updated orders
        where userOrders = searchOrders [ById id] currentAllOrders

: 

print orders of user with loyalty level `ll`, and process and ship all the pending orders

In [51]:
ordersByLL :: LoyaltyLevel -> [Order] -> IO [Order]
ordersByLL ll currentAllOrders
        | null usersOrders = do
                putStrLn "No orders by this Loyalty Level"
                searchByLL currentAllOrders
        | otherwise = do
                        putStrLn "Orders:"
                        putStrLn $ listToString usersOrders
                        newAllOrders <- return $ updateOrders usersOrders currentAllOrders Shipped
                        putStrLn "Orders updated"
                        start newAllOrders
        where usersOrders = searchOrders [ByLoyaltyLevel ll] currentAllOrders

Line 9: Use let
Found:
newAllOrders <- return
                  $ updateOrders usersOrders currentAllOrders Shipped
Why not:
let newAllOrders
      = updateOrders usersOrders currentAllOrders Shipped

: 

Process and ship all the pending orders of high value customers with `limit` total price of all active orders

In [52]:
ordersByHighValue :: [Order] -> [Customer] -> [Order]
ordersByHighValue currentAllOrders customers = updateOrders customersOrders pendingToProcessing Shipped
        where   customersOrders = concat [ searchOrders [ById id] currentAllOrders | (Customer id _ _) <- customers ]
                pendingToProcessing = updateOrders customersOrders currentAllOrders Processing


Check user `id` provided for being correct and process further

In [53]:
checkIdInput :: String -> [Order] -> IO [Order]
checkIdInput id currentAllOrders
        | all isDigit id = ordersById (read id :: Int) currentAllOrders
        | id == "b" || id == "B" = startWork currentAllOrders
        | otherwise = do
                putStrLn "ID consist only of digits!"
                searchById currentAllOrders


searchById :: [Order] -> IO [Order]
searchById currentAllOrders = do
        putStrLn "Introduce user ID (b to come back):"
        id <- getLine
        checkIdInput id currentAllOrders

: 

Check loyalty level `ll` provided for being correct and process further

In [54]:
searchByLL :: [Order] -> IO [Order]
searchByLL currentAllOrders = do
        putStrLn "Introduce user Loyalty Level (Bronze, Silver or Gold. b to come back): "
        ll <- getLine
        case (map toLower ll) of
        -- data LoyaltyLevel = Bronze | Silver | Gold deriving (Show, Eq)
            "bronze" -> ordersByLL Bronze currentAllOrders
            "silver" -> ordersByLL Silver currentAllOrders
            "gold"   -> ordersByLL Gold currentAllOrders
            "b"      -> startWork currentAllOrders
            _        -> do 
                    putStrLn "There is no such Loyalty Level"
                    searchByLL currentAllOrders

Line 5: Redundant bracket
Found:
case (map toLower ll) of
  "bronze" -> ordersByLL Bronze currentAllOrders
  "silver" -> ordersByLL Silver currentAllOrders
  "gold" -> ordersByLL Gold currentAllOrders
  "b" -> startWork currentAllOrders
  _ -> do putStrLn "There is no such Loyalty Level"
          searchByLL currentAllOrders
Why not:
case map toLower ll of
  "bronze" -> ordersByLL Bronze currentAllOrders
  "silver" -> ordersByLL Silver currentAllOrders
  "gold" -> ordersByLL Gold currentAllOrders
  "b" -> startWork currentAllOrders
  _ -> do putStrLn "There is no such Loyalty Level"
          searchByLL currentAllOrders

: 

Checks `limit` provided for being proper number and process further

In [55]:
usersByHighValue :: Float -> [Order] -> IO [Order]
usersByHighValue limit currentAllOrders
        | limit > 0 = do
                putStrLn "Orders updated"
                start $ ordersByHighValue currentAllOrders (highValueCustomers currentAllOrders limit)
        | otherwise = do
                putStrLn "Total price should be non negative number"
                searchByHighValue currentAllOrders
                

searchByHighValue :: [Order] -> IO [Order]
searchByHighValue currentAllOrders = do
        putStrLn "Introduce the minimum total price of not Delivered or Cancelled orders (b to come back):"
        value <- getLine
        case readMaybe value :: Maybe Float of
                Just number -> usersByHighValue number currentAllOrders
                Nothing     -> if value == "b" || value == "B" then startWork currentAllOrders
                        else do
                        putStrLn "That is not a number"
                        searchByHighValue currentAllOrders

: 

Offers options to select orders

In [56]:
startWork :: [Order] -> IO [Order]
startWork currentAllOrders = do 
        putStrLn "What do you want to do?"
        putStrLn "Options:\n1. Search orders by user ID\n2. Search orders by Loyalty Level"
        putStrLn "3. Search orders by high value customers"
        answer <- getLine
        case answer of
            "1" -> searchById currentAllOrders
            "2" -> searchByLL currentAllOrders
            "3" -> searchByHighValue currentAllOrders
            "b" -> start currentAllOrders
            _   -> do
                    putStrLn "There is no such option"
                    startWork currentAllOrders

: 

Handles start input

In [57]:
checkStartAnswer :: String -> [Order] -> IO [Order]
checkStartAnswer answer currentAllOrders
        | answer `elem` ["yes", "yeah", "y"] = startWork currentAllOrders
        | answer `elem` ["no", "nope", "n", "nah"] = do
                putStrLn "Understood :("
                return currentAllOrders
        | otherwise = do
                    putStrLn "I do not understand you"
                    start currentAllOrders


start :: [Order] -> IO [Order]
start currentAllOrders = do
        putStrLn "Do you want to continue?"
        answer <- getLine
        checkStartAnswer (map toLower answer) currentAllOrders

: 

Starts session, load orders and later save them back to file

In [58]:
session :: IO ()
session = do
        putStrLn "Welcome"
        putStrLn "\nReading orders..."
        loadedOrders <- readOrdersFromFile "orders.txt"
        putStrLn "Orders loaded:"

        allorders <- start loadedOrders
        putStrLn "Bye, have a good day!"
        -- Save orders
        putStrLn "Saving orders..."
        writeOrdersToFile "orders.txt" allorders
        putStrLn "Orders saved!"

: 